# Priority Queue ADT

In situations where a "first come, first serve" policy appears reasonable but additional priorities come into play, a priority queue becomes essential. A priority queue is an abstract data type that manages a collection of prioritized elements, allowing arbitrary element insertion and removal of the element with the highest priority. Each element added to the priority queue is assigned a priority key by the user.

The element with the minimum key (or highest priority) is dequeued next. Although keys are often expressed numbers, any object can serve as a key as long as it supports a consistent meaning for the less-than comparison (a < b) to establish a natural order.

## The Priority Queue ADT

Formally, we model an element and its priority as a key-value pair. We define the
priority queue ADT to support the following methods for a priority queue P:

- P.add(k, v): Insert an item with key k and value v into priority queue P.
- P.min(): Return a tuple, (k,v), representing the key and value of an item in priority queue P with minimum key (but do not remove the item); an error occurs if the priority queue is empty.
- P.remove_min( ): Remove an item with minimum key from priority queue P, and return a tuple, (k,v), representing the key and value of the removed item; an error occurs if the priority queue is empty.
-  P.is empty( ): Return True if priority queue P does not contain any items.
- len(P): Return the number of items in priority queue P.

A priority queue may have multiple items with equivalent keys, in which case the methods min and remove_min may return an arbitrary choice of item having minimum key, or the one that was inserted first.

In [1]:
from abc import ABC, abstractmethod

class PriorityQueueBase(ABC):
    class _Item:
        __slots__ = '_key', '_value'

        def __init__(self,k,v):
            self._key = k
            self._value = v
        
        def __lt__(self, other):
            return self._key < other._key
    
    @abstractmethod
    def add(self, k, v):
        raise NotImplementedError()

    @abstractmethod
    def min(self):
        raise NotImplementedError()

    @abstractmethod
    def __len__(self):
        raise NotImplementedError()

    @abstractmethod
    def remove_min(self):
        raise NotImplementedError()
    
    def is_empty(self):
        """Returns True if thr priority queue is empty"""
        return len(self) == 0

### Implementation with an unsorted List

We use a doubly linked list to keep track of the elements in the priority queue.

In [2]:
from collections import deque

class UnsortedListPriorityQueue(PriorityQueueBase):
    def __init__(self) -> None:
        super().__init__()
        self.pq = deque()

    def __len__(self):
        """Return the number of elements in the priority queue"""
        return len(self.pq)
    
    def add(self, key, value):
        """Add a key value pair to the list"""
        self.pq.append(self._Item(key, value))
    
    def min(self):
        p = min(self.pq)
        return (p._key, p._value)
    
    def remove_min(self):
        p = min(self.pq)
        self.pq.remove(p)
        return (p._key, p._value)
    
pq = UnsortedListPriorityQueue()
pq.add(10,'Alfredo')
pq.add(15,'Maria')
pq.add(19,'Sara')
pq.add(10,'Pedro')
pq.add(17,'Ursula')
pq.add(10,'Elemer')

while pq:
    print(pq.remove_min())

(10, 'Alfredo')
(10, 'Pedro')
(10, 'Elemer')
(15, 'Maria')
(17, 'Ursula')
(19, 'Sara')


#### Analysis:
- len: $O(1)$
- is_empty: $O(1)$
- add: $O(1)$
- min: $O(n)$
- remove_min: $O(n)$

### Implementation with a sorted List

A regular array is enough to store all the elements.

In [3]:
class UnsortedListPriorityQueue(PriorityQueueBase):
    def __init__(self) -> None:
        super().__init__()
        self.pq = list()

    def __len__(self):
        """Return the number of elements in the priority queue"""
        return len(self.pq)
    
    def add(self, key, value):
        """Add a key value pair to the list"""
        item = self._Item(key, value)
        
        idx = 0
        while idx < len(self) and self.pq[idx] > item:
            idx += 1
        
        self.pq.insert(idx, item)

    
    def min(self):
        if self.is_empty():
            raise Exception("The pq is empty")
        item = self.pq[-1]
        return (item._key, item._value)
    
    def remove_min(self):
        if self.is_empty():
            raise Exception("The pq is empty")
        item = self.pq.pop()
        return (item._key, item._value)
    
pq = UnsortedListPriorityQueue()
pq.add(10,'Alfredo')
pq.add(15,'Maria')
pq.add(19,'Sara')
pq.add(10,'Pedro')
pq.add(17,'Ursula')
pq.add(10,'Elemer')

while pq:
    print(pq.remove_min())

(10, 'Alfredo')
(10, 'Pedro')
(10, 'Elemer')
(15, 'Maria')
(17, 'Ursula')
(19, 'Sara')


#### Analysis:
- len: $O(1)$
- is_empty: $O(1)$
- add: $O(n)$
- min: $O(1)$
- remove_min: $O(1)$

## Comparaison:

The two strategies for implementing a priority queue ADT above demonstrate an interesting trade-off. When using an unsorted list to store entries,
we can perform insertions in $O(1)$ time, but finding or removing an element with minimum key requires an $O(n)$-time loop through the entire collection. In contrast,
if using a sorted list, we can trivially find or remove the minimum element in $O(1)$ time, but adding a new element to the queue may require $O(n)$ time to restore the
sorted order.

# The Heap Data Structure

<center>
<figure align = ="center">
<img  src=images/heap.png style="width:40%">
<figcaption align = "center"> An example of a heap </figcaption>
</figure>
</center>

A heap is a binary tree T that stores a collection of items at its positions and that satisfies two additional properties:
- **Heap-Order Property**: In a heap T, for every position p other than the root, the key stored at p is greater than or equal to the key stored at p’s parent.
- **Complete Binary Tree Property**: A heap T with height h is a complete binary tree if levels 0,1,2,...,h− 1 of T have the maximum number of nodes possible (namely, level i has 2i nodes, for 0 ≤ i ≤ h− 1) and the remaining nodes at level h reside in the leftmost possible positions at that level.





### Why?
The heap provides a more efficient realization of a priority queue using
a data structure, it allows us to perform both insertions and removals in logarithmic time, which is a significant improvement
over the list-based implementations discussed above. The fundamental way the heap achieves this improvement is to use the structure of a binary tree to find a compromise between elements being entirely unsorted and perfectly sorted.


**Why impose the heap order property?**

As a consequence of the heap-order property, the keys encountered on a path from the root to a leaf of T are in nondecreasing order. Also, a minimum key is always stored at the root of T.

**Why impose the complete property?**

For the sake of efficiency, as will become clear later, we want the heap T to have as small a height as possible. We enforce this requirement by insisting that the heap T satisfy an additional structural property—it must be what we term complete.

## Properties

A heap storing n elements has a height h given by:
$$h = \lfloor{log(n)}\rfloor$$

Proof: The maximum number of elements that a tree of height h can store is given by:

$$n = \sum_{i=0}^{h-1}{2^i} = 2^h - 1$$

And the statement above is equivalent to:
$$h <= \log{n} <= h +1$$
So:

$$ n >= 2^{h} -1 + 1 = 2^{h}$$
$$ n <= 2^{h+1} -1$$
Which implies:
$$  2^{h} <= n<= 2^{h+1} -1 <= 2^{h+1}$$

Taking logarighms:

$$ h <= \log{n} <= \log{2^{h+1}-1} <= h +1 $$ 

## Inserting and element: Percolating up or bubbling up

To insert an element into the heap, we first place it into the first available position that satisfies the structural property of the heap.

To make sure that we still have a valid heap, we will need to update some of its nodes to make sure we verify the order property, we can do that by swapping the inserted node node with its parent repeatedly until the order property is restored. 

The time complexity of this operation, is, in the worst case, $O(\log(n))$

## Deleting an element: Percolating down or bubbling down

To delete an element from the heap, we first do the following swap: The element at the top of the heap is put in the place of the element that needs to be deleted, the element at the rightmost position of the heap is put in the place of the element that needs to be removed. This way, the element is removed from the heap without breaking the structural property, but the order property is broken.

To restore the order property, the element at the top of the heap (which previously was the one at the rightmost-bottom position) is replaced with either its right or left child (whichever has the lowest key) until the order property is restored.

The time complexity of this operation, is, in the worst case, $O(\log(n))$

## Using an array list to represent a heap

The array-based representation of a binary tree is especially suitable for a complete binary tree T. In particular, we will use an array list with a None element stored at index 0. In that case, the following properties are satisfied:

For the i-th node:
- The index of its parent is: $ parent = \lfloor i/2 \rfloor$
- The index of its left child is: $ lchild = 2*i$
- The index of its right child is $rchild = 2*i + 1$


An additional property that will be useful in the heapify algorithm is that all the leaves of the tree are located in the last $\lfloor n/2 \rfloor$ positions of the array

## Heap heapify: Building a heap in linear time

Building a heap an element at a time has a worst case time coplexity of $n\log(n)$. If we already know the elemens that make up the heap we can do better. 

To do so, we start by initializing the heap array with the input elements. This way we are meeting the structural propery but not the order property. To fix the heap, we could either percolate up all the nodes in the tree, or we could percolate down all the non leave nodes (as percolating down the leaves does not do anything)

It is clearly more efficient to percolate down, since most of our leaves will be near the base of the tree and they will only need to move a few places, while percolating up requires most of the leaves to require almost the full depth of the tree

The worst case time complexity of this algorithm is given by:

$$ (n+1)/4 + 2(n+1)/8 + 3(n+1)/16 + ... = \sum_{i=1}^{h} \frac{(n+1)\cdot i}{2\cdot2^{i}} = \frac{(n+1)}{2}\sum_{i=1}^{h} \frac{i}{2^{i}}  <= (n+1)$$

Since at most quareter of the leaves need to be moved at most 1 level, one eight of the leaves need to be moved 2 leaves etc

#### Analysis:
- len: $O(1)$
- is_empty: $O(1)$
- add: $O(\log(n))$
- min: $O(1)$*
- remove_min: $O(\log(n))$*

\*amortized time

In [17]:
class Heap:
    class _Item:
        __slots__ = '_key', '_value'

        def __init__(self,k,v):
            self._key = k
            self._value = v
        
        def __lt__(self, other):
            return self._key < other._key

        def __le__(self, other):
            return self._key <= other._key

        def __gt__(self, other):
            return self._key > other._key

        def __ge__(self, other):
            return self._key >= other._key
        
        def __eq__(self, other):
            return self._key == other._key
    
    def __init__(self):
        self._data = [None]

    @classmethod
    def heapify(cls, input_data):
        h = cls()
        h._data = [None]
        
        for (k,v) in input_data:
            h._data.append(h._Item(k,v))

        i = len(h)//2 
        while i> 0:
            h.percolate_down(i)
            i -= 1

        return h
    
    def __len__(self):
        return len(self._data) - 1

    def _lchild(self, i: int) -> int:
        return 2*i

    def _rchild(self, i: int) -> int:
        return 2*i+1

    def _parent(self, i: int) -> int:
        return i//2
    
    def _swap(self, i: int, j: int) -> None:
        self._data[i], self._data[j] = self._data[j], self._data[i]

    def percolate_up(self, i: int) -> None:
        while i > 1:
            p = self._parent(i)
            if self._data[i] < self._data[p]:
                self._swap(i, p)
            else:
                break
            i = p
    
    def percolate_down(self, i: int) -> None:
        
        while i < len(self):
            r = self._rchild(i)
            l = self._lchild(i)
            if r < len(self) and self._data[i] > self._data[r] and self._data[r] <= self._data[l]:
                self._swap(i, r)
                i = r
            elif l < len(self) and self._data[i] > self._data[l]:
                self._swap(i, l)
                i = l
            else:
                break

    def min(self):
        return (self._data[1]._key, self._data[1]._value)
    
    def push(self, k, v):
        self._data.append(self._Item(k,v))
        self.percolate_up(len(self))
    
    def pop(self):
        if len(self) == 0:
            raise Exception("The heap is empty")
        self._swap(1, len(self))
        item = self._data.pop()
        self.percolate_down(1)
        return (item._key, item._value)
    


pq = Heap()
pq.push(17,'Ursula')
pq.push(10,'Alfredo')
pq.push(15,'Maria')
pq.push(19,'Sara')
pq.push(10,'Pedro')


while pq:
    print(pq.pop())

(10, 'Alfredo')
(10, 'Pedro')
(17, 'Ursula')
(15, 'Maria')
(19, 'Sara')


## Heap Sort

In [31]:
from random import randint

def heap_sort(data):
    pq = Heap.heapify([(x,x) for x in data])

    i = 0
    while len(pq) > 0:
        data[i] = pq.pop()[0]
        i += 1


sample_input = [randint(0,100) for _ in range(10)]

print(f"Input: {sample_input}")
heap_sort(sample_input)
print(f"Sorted: {sample_input}")

Input: [91, 42, 62, 11, 59, 98, 84, 9, 79, 80]
Sorted: [9, 11, 42, 59, 62, 79, 80, 91, 84, 98]
